In [1]:
import cv2
import numpy as np

CASCADE_PATH = "haarcascade_frontalface_default.xml"

RESIZE_SCALE = 3
REC_COLOR = (0, 255, 0)

def getFaceCoordinates(image):
    cascade = cv2.CascadeClassifier(CASCADE_PATH)
    
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.equalizeHist(img_gray)
    rects = cascade.detectMultiScale(
        img_gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(48, 48)
        )

    # For now, we only deal with the case that we detect one face.
    if(len(rects) != 1) :
        return None
    
    face = rects[0]
    bounding_box = [face[0], face[1], face[0] + face[2], face[1] + face[3]]

    # return map((lambda x: x), bounding_box)
    return bounding_box

def drawFace(img, faceCoordinates):
    cv2.rectangle(np.asarray(img), (faceCoordinates[0], faceCoordinates[1]), \
    (faceCoordinates[2], faceCoordinates[3]), REC_COLOR, thickness=2)

def crop_face(img, faceCoordinates):
    '''
    extend_len_x =  (256 - (faceCoordinates[3] - faceCoordinates[1]))/2
    extend_len_y =  (256 - (faceCoordinates[0] - faceCoordinates[2]))/2
    img_size = img.shape
    if (faceCoordinates[1] - extend_len_x) >= 0 :
        faceCoordinates[1] -= extend_len_x
    if (faceCoordinates[3] + extend_len_x) < img_size[0]:
        faceCoordinates[3] += extend_len_x
    if (faceCoordinates[0] - extend_len_y) >= 0 :
        faceCoordinates[0] -= extend_len_y
    if (faceCoordinates[2] + extend_len_y) < img_size[1]:
        faceCoordinates[2] += extend_len_y
    '''
    return img[faceCoordinates[1]:faceCoordinates[3], faceCoordinates[0]:faceCoordinates[2]]

def preprocess(img, faceCoordinates, face_shape=(48, 48)):
    '''
        This function will crop user's face from the original frame
    '''
    face = crop_face(img, faceCoordinates)
    #face = img
    face_scaled = cv2.resize(face, face_shape)
    face_gray = cv2.cvtColor(face_scaled, cv2.COLOR_BGR2GRAY)
    
    return face_gray

In [ ]:
import argparse
import sys, os
sys.path.append("../")

import cv2
import numpy as np

import face_detection_utilities as fdu

import model.myVGG as vgg
import speech_recognition as sr

windowsName = 'Preview Screen'

parser = argparse.ArgumentParser(description='A live emotion recognition from webcam')

args = parser.parse_args()
FACE_SHAPE = (48, 48)

model = vgg.VGG_16('my_model_weights_83.h5')
#model = vgg.VGG_16()

emo     = ['Angry', 'Fear', 'Happy',
           'Sad', 'Surprise', 'Neutral']

def speechRecognition():
    # obtain audio from the microphone 
    print("Press 'y' to start~")
    inputdata = input()
    if inputdata == 'y':
        inputdata = 0
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Say something!")
            audio = r.listen(source)
        # recognize speech using Google Speech Recognition
        try:
        # for testing purposes, we're just using the default API key
        # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
        # instead of `r.recognize_google(audio)`
            recSuccess = 1
            recContent = r.recognize_google(audio)
            print("Speech Recognition thinks you said " + recContent)#,language="cmn-Hant-TW")
            return recContent
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))

def refreshFrame(frame, faceCoordinates):
    if faceCoordinates is not None:
        fdu.drawFace(frame, faceCoordinates)
    cv2.imshow(windowsName, frame)


def showScreenAndDectect(capture):
    print("Face dececting...")
    cnt = 5;
    while cnt:
        flag, frame = capture.read()
        faceCoordinates = fdu.getFaceCoordinates(frame)
        refreshFrame(frame, faceCoordinates)
        
        if faceCoordinates is not None:
            cnt -= 1
            face_img = fdu.preprocess(frame, faceCoordinates, face_shape=FACE_SHAPE)
            #cv2.imshow(windowsName, face_img)

            input_img = np.expand_dims(face_img, axis=0)
            input_img = np.expand_dims(input_img, axis=0)

            result = model.predict(input_img)[0]
            if cnt == 4:
                tot_result = result
            else:
                tot_result += result
            index = np.argmax(result)
            print ('Frame',5-cnt,':', emo[index], 'prob:', max(result))
            #index = np.argmax(result)
            #print (emo[index], 'prob:', max(result))
            # print(face_img.shape)
            # emotion = class_label[result_index]
            # print(emotion)
    index = np.argmax(tot_result)
    print ('Final decision:',emo[index], 'prob:', max(tot_result))
    return emo[index]

def getCameraStreaming():
    capture = cv2.VideoCapture(0)
    if not capture:
        print("Failed to capture video streaming ")
        sys.exit(1)
    else:
        print("Successed to capture video streaming")
        
    return capture

def main():
    '''
    Arguments to be set:
        showCam : determine if show the camera preview screen.
    '''
    print("Enter main() function")
    
    capture = getCameraStreaming()

    cv2.startWindowThread()
    cv2.namedWindow(windowsName, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(windowsName, cv2.WND_PROP_FULLSCREEN, cv2.WND_PROP_FULLSCREEN)
    
    while True:
        recContent = speechRecognition()
        if recContent is not None:
            emotion = showScreenAndDectect(capture)
            if emotion == "Angry":
                emoji = " >:O"
            elif emotion == "Fear":
                emoji = " :-S"
            elif emotion == "Happy":
                emoji = " :-D"
            elif emotion == "Sad":
                emoji = " :'("
            elif emotion == "Surprise":
                emoji = " :-O"
            else:
                emoji = " "
            print("Output result: " + recContent + emoji)

if __name__ == '__main__':
    main()
